In [1]:
# Import the following libraries
import requests
import folium
import folium.plugins
from folium import Map, TileLayer
from pystac_client import Client
import branca
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import branca.colormap as cm
import seaborn as sns

/Users/yuzu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
!pip3 install --upgrade gradio

In [ ]:
!pip3 install geopy

In [2]:
# Provide the STAC and RASTER API endpoints
# The endpoint is referring to a location within the API that executes a request on a data collection nesting on the server.

# The STAC API is a catalog of all the existing data collections that are stored in the GHG Center.
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"

# The RASTER API is used to fetch collections for visualization
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
# Name of the collection for Vulcan Fossil Fuel CO₂ Emissions, Version 4. 
collection_ffco2 = "odiac-ffco2-monthgrid-v2023"
# Name of the collection for MiCASA Land Carbon Flux
collection_landcarbon = "micasa-carbonflux-daygrid-v1"
# Name of the collection for methane emission plumes 
collection_ch4plume = "emit-ch4plume-v1"
# Name of the collection for CEOS National Top-Down CO₂ Budgets dataset 
collection_co2budget = "oco2-mip-co2budget-yeargrid-v1"

In [3]:
collection_ffco2 = requests.get(f"{STAC_API_URL}/collections/{collection_ffco2}").json()
collection_landcarbon = requests.get(f"{STAC_API_URL}/collections/{collection_landcarbon}").json()
collection_ch4plume = requests.get(f"{STAC_API_URL}/collections/{collection_ch4plume}").json()
collection_co2budget = requests.get(f"{STAC_API_URL}/collections/{collection_co2budget}").json()

In [4]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [14]:
num_items_ffco2 = get_item_count(collection_ffco2["id"])

items_ffco2 = requests.get(f"{STAC_API_URL}/collections/{collection_ffco2['id']}/items?limit={num_items_ffco2}").json()["features"]

In [15]:
# num_items_landcarbon = get_item_count(collection_landcarbon["id"])
num_items_landcarbon = 800

items_landcarbon = requests.get(f"{STAC_API_URL}/collections/{collection_landcarbon['id']}/items?limit={num_items_landcarbon}").json()["features"]

In [16]:
num_items_ch4plume = 1493

items_ch4plume = requests.get(f"{STAC_API_URL}/collections/{collection_ch4plume['id']}/items?limit={num_items_ch4plume}").json()["features"]

In [17]:
# Apply the function created above "get_item_count" to the data collection
num_items_co2budget = 6

# Get the information about the number of granules found in the collection
items_co2budget = requests.get(f"{STAC_API_URL}/collections/{collection_co2budget['id']}/items?limit={num_items_co2budget}").json()["features"]


In [18]:
items_ffco2 = {item["properties"]["start_datetime"][:7]: item for item in items_ffco2}
items_landcarbon = {item["properties"]["datetime"][:10]: item for item in items_landcarbon}
items_ch4plume = {item["id"]: item for item in items_ch4plume}
items_co2budget = {item["properties"]["start_datetime"]: item for item in items_co2budget}

asset_ffco2 = "co2-emissions"
asset_landcarbon = "rh"
assert_ch4plume = "ch4-plume-emissions"
asset_co2budget = "ff"

In [19]:
rescale_ffco2 = {"max":items_ffco2[list(items_ffco2.keys())[0]]["assets"][asset_ffco2]["raster:bands"][0]["histogram"]["max"], "min":items_ffco2[list(items_ffco2.keys())[0]]["assets"][asset_ffco2]["raster:bands"][0]["histogram"]["min"]}
rescale_landcarbon = {"max":items_landcarbon[list(items_landcarbon.keys())[0]]["assets"][asset_landcarbon]["raster:bands"][0]["histogram"]["max"], "min":items_landcarbon[list(items_landcarbon.keys())[0]]["assets"][asset_landcarbon]["raster:bands"][0]["histogram"]["min"]}
rescale_ch4plume = {"max":items_ch4plume[list(items_ch4plume.keys())[0]]["assets"][assert_ch4plume]["raster:bands"][0]["histogram"]["max"], "min":items_ch4plume[list(items_ch4plume.keys())[0]]["assets"][assert_ch4plume]["raster:bands"][0]["histogram"]["min"]}
rescale_co2budget = {"max":items_co2budget[list(items_co2budget.keys())[0]]["assets"][asset_co2budget]["raster:bands"][0]["histogram"]["max"], "min":items_co2budget[list(items_co2budget.keys())[0]]["assets"][asset_co2budget]["raster:bands"][0]["histogram"]["min"]}
rescale_co2budget = {"max": 450, "min": 0}

In [20]:
from geopy.geocoders import Nominatim

def search_location(location):
    geolocator = Nominatim(user_agent="user-id")
    location = geolocator.geocode(location)
    if location:
        return location.latitude, location.longitude
    else:
        # print(f"Error: '{city}' not found. Using default location (Tokyo).")
        return 35.682839, 139.759455  # 東京の緯度・経度

In [47]:
import random
from datetime import datetime, timedelta
from branca.colormap import LinearColormap

color_map = "spectral_r"

def get_valid_tile_url(date, item_dict, asset, rescale, color_map):
    """
    Check if the data exists on the specified date, and if not, move to the next day to search for data.
    """
    while date <= datetime.now():  # Loop until the current date
        if asset == "rh":
            formatted_date = date.strftime("%Y-%m-%d")
        elif asset == "co2-emissions":
            formatted_date = date.strftime("%Y-%m")
        elif asset == "ff":
            formatted_date = date.strftime("%Y")
            year = int(formatted_date)
            if year > 2020 or year < 2015:
                return None
            now = 2020
            i = 0
            while now != year:
                i += 1
                now -= 1
            formatted_date = list(item_dict.keys())[i]
        try:
            tile_url = requests.get(
                f"{RASTER_API_URL}/collections/{item_dict[formatted_date]['collection']}/items/{item_dict[formatted_date]['id']}/tilejson.json?"
                f"&assets={asset}"
                f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
                f"&rescale={rescale['min']},{rescale['max']}", 
            ).json()
            return tile_url
        except KeyError:
            # If there is no data on the date, proceed to the next day
            date += timedelta(days=1)
    
    print(f"Error: Data not found for {date.strftime('%Y-%m-%d')}.")
    return None

def year_map(city="tokyo", year=2022, random_search=False):
    year = str(year)
    
    # Get ffco2 tile
    ffco2_date = datetime.strptime(f"{year}-01", "%Y-%m")
    _ffco2_tile = get_valid_tile_url(ffco2_date, items_ffco2, asset_ffco2, rescale_ffco2, color_map)
    
    # Acquisition of Landcarbon tile
    landcarbon_date = datetime.strptime(f"{year}-01-01", "%Y-%m-%d")
    _landcarbon_tile = get_valid_tile_url(landcarbon_date, items_landcarbon, asset_landcarbon, rescale_landcarbon, color_map)
    
    # Acquisition of co2budget tile
    co2budget_date = datetime.strptime(f"{year}-01-01", "%Y-%m-%d")
    _co2budget_tile = get_valid_tile_url(co2budget_date, items_co2budget, asset_co2budget, rescale_co2budget, color_map)
    
    # Acquire latitude and longitude based on the city name
    lat, lng = search_location(city)

    # Create Folium DualMap
    map_ = folium.plugins.DualMap(location=(lat, lng), zoom_start=6)
    map__ = folium.Map(location=(lat, lng), zoom_start=6)
    
    # Creating a color map
    colormap = LinearColormap(
        colors=['#310597', '#4C02A1', '#6600A7', '#7E03A8', '#9511A1', '#AA2395', '#BC3587', '#CC4778', '#DA5A6A', '#E66C5C', '#F0804E', '#F89540','#FDAC33', '#FDC527', '#F8DF25'],
        vmin=0, vmax=1500
    )
    
    if random_search:
        colormap.caption = 'ppm-m'
        
        ppm_value = colormap((rescale_ch4plume['max'] - rescale_ch4plume['min']) / 2)

        # Obtain 15 random keys randomly based on items_ch4plume
        random_keys = random.sample(list(items_ch4plume.keys()), min(15, len(items_ch4plume)))
        
        for key in random_keys:
            methane_plume_tile = requests.get(
                f"{RASTER_API_URL}/collections/{items_ch4plume[key]['collection']}/items/{items_ch4plume[key]['id']}/tilejson.json?"
                f"&assets={assert_ch4plume}"
                f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
                f"&rescale={rescale_ch4plume['min']},{rescale_ch4plume['max']}", 
            ).json()
            
            # Added to both creation of markers and DualMap
            for map_instance in [map_.m1, map_.m2]:
                folium.Marker(
                    location=[methane_plume_tile["center"][1], methane_plume_tile["center"][0]],
                    popup=items_ch4plume[key]["id"],
                    icon=folium.Icon(color=ppm_value, icon="cloud"),
                ).add_to(map_instance)
        
    if _ffco2_tile:
        map_layer_ffco2 = TileLayer(
            tiles=_ffco2_tile["tiles"][0],
            attr="GHG",
            name=f'{year} Total CO2 Fossil Fuel Emissions',
            overlay=True,
            opacity=0.8,
        )
        map_layer_ffco2.add_to(map_.m1)
    
    if _landcarbon_tile:
        map_layer_landcarbon = TileLayer(
            tiles=_landcarbon_tile["tiles"][0],
            attr="GHG",
            name=f'{year} RH Level',
            overlay=True,
            opacity=0.8,
        )
        map_layer_landcarbon.add_to(map_.m2)
        
    if _co2budget_tile:
        map_layer_co2budget = TileLayer(
            tiles=_co2budget_tile["tiles"][0],
            attr="GHG",
            name=f'{year} National Top-Down CO2 Budgets',
            overlay=True,
            opacity=0.8,
        )
        map_layer_co2budget.add_to(map__)

    return map_._repr_html_() + map__._repr_html_()

In [52]:
import gradio as gr


custom_css = """
body {
    font-family: Inter, Public Sans, sans-serif;
    font-size: 20px !important;
}

.nasa_logo img {
    margin-left: 50px;
    background-color: black;
}

.logo img {
    margin-right: 50px;
}

.header-container h1 {
    font-size: 40px !important;  /* Specify the font size of the title */
    text-align: left;           /* The title is aligned center */
    margin-top: 50px;          /* Add space on the title */
}

.markdown h1 {
    font-size: 400px !important;
    margin-top: 100px;         
}

.markdown p {
    font-size: 50px !important;  /* Specify the font size of the paragraph */
}

.map-title h3 {
    text-align: center;
    font-size: 20px;
}

.fadeout img {
    position: fixed;
    top: 0;
    left: 0;
    width: 100vw;  /* 100% width */
    height: 100vh; /* 100% height */
    object-fit: cover !important; /* As the image covers the area */
    z-index: 100; /* z-index for placing it in the background */
    transition: opacity 3s ease-out;  /* Fade -out animation */
}
"""

custom_js = """
function fadeout(duration = 3000, delay = 1500) {  
    const fadeOutElement = document.getElementById('fadeout');
    
    if (fadeOutElement) {
        fadeOutElement.style.transition = `opacity ${duration}ms ease-out`;  // Set the fade-out time
        setTimeout(() => {
            fadeOutElement.style.opacity = '0';  // fade-out
            setTimeout(() => {
                fadeOutElement.style.display = 'none';  // Hidden after the specified time
                console.log('Element faded out');
            }, duration); // Hidden after the specified time
        }, delay); // Fade out starts after the specified delay time
    }
    console.log('Fadeout function executed');
    return true;
}

"""

# Creating Gradio interface
with gr.Blocks(theme=gr.themes.Default(primary_hue="cyan", secondary_hue="yellow"), css=custom_css, js=custom_js, head=custom_js) as demo:
    
    # Image to fade-out
    gr.Image('./../images/fadeout.png', elem_id="fadeout", elem_classes="fadeout", container=False, show_download_button=False, show_fullscreen_button=False)

    # Insert images and titles in the header
    with gr.Row():
        gr.Image('./../images/nasa_space_apps.png', elem_classes="nasa_logo", width="150px", height="150px", container=False, show_download_button=False, show_fullscreen_button=False)
        gr.Image('./../images/logo.png', elem_classes="logo", width="150px", height="150px", container=False, show_download_button=False, show_fullscreen_button=False)
    
    gr.Markdown("""
    <h2><p>This map shows the human-caused and natural greenhouse gas emissions data over the years. The left map shows the fossil fuel CO2 emissions and the right map shows the natural CO2 emissions.</p></h2>
    """)
    
    # Search the location in a text box
    city_input = gr.Textbox(label="Enter a location to search for GHG emission data:", placeholder="Type location name...", lines=1)
    
    random_search = gr.Checkbox(label="Show large methane emissions", value=False)
    
    # Show the map
    output = gr.HTML()
    
    # Place slider (search bar)
    year_slider = gr.Slider(2001, 2022, step=1, label="Select Year", interactive=True)
    
    city_input.submit(year_map, inputs=[city_input, year_slider, random_search], outputs=output)
    year_slider.change(year_map, inputs=[city_input, year_slider, random_search], outputs=output)

demo.launch()

Running on local URL:  http://127.0.0.1:7893

To create a public link, set `share=True` in `launch()`.
